In [3]:
import cv2 
import os
from PIL import Image
import numpy as np

In [4]:
image_directory='dataset/'
no_tumor_images=os.listdir(image_directory+ 'no/')
yes_tumor_images=os.listdir(image_directory+ 'yes/')
dataset=[]
label=[]
INPUT_SIZE=64




In [5]:
# checking image is jpg or not and making dataset and label
for i , image_name in enumerate(no_tumor_images):
    if(image_name.split('.')[1]=='jpg'):
        image=cv2.imread(image_directory+'no/'+image_name)
        image=Image.fromarray(image, 'RGB')
        image=image.resize((INPUT_SIZE,INPUT_SIZE)) 
        dataset.append(np.array(image))
        label.append(0)
for i , image_name in enumerate(yes_tumor_images):
    if(image_name.split('.')[1]=='jpg'):
        image=cv2.imread(image_directory+'yes/'+image_name)
        image=Image.fromarray(image, 'RGB')
        image=image.resize((INPUT_SIZE,INPUT_SIZE)) 
        dataset.append(np.array(image))
        label.append(1)

In [6]:
# dataset and label 
print(len(dataset))
print(len(label))

3000
3000


In [7]:
#changing data into numpy array
dataset=np.array(dataset)
label=np.array(label)

In [8]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(dataset,label,test_size=0.2,random_state=0)

In [9]:
import tensorflow as tf
from tensorflow import keras
from keras.utils import normalize


In [10]:
# normalizing data for binary classification
x_train=normalize(x_train, axis=1)
x_test=normalize(x_test, axis=1)

In [11]:
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D
from keras.layers import Activation,Dropout,Flatten,Dense

In [12]:
# model building

#sequential model
model=Sequential()
#firstlayer: putting shape of the input
#Conv2D(filter,kersnal_size)using 32 no. of convolution and
# using (3,3) kernal size and input size of 64x64
model.add(Conv2D(32,(3,3), input_shape=(INPUT_SIZE,INPUT_SIZE,3)))
#adding activation fuction
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

#initializing kernel
model.add(Conv2D(32,(3,3), kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Conv2D(64,(3,3), kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

#flattening
model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))

#output layer using 1 coz using binary classification prob 
#if we used categorical classification we would have used 2
model.add(Dense(1))
#using sigmoid function as binary cross entropy 
#if categorical cross entropy we use softmax
model.add(Activation('sigmoid'))

#compiling the model using binary_cross_entrpy
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),optimizer='adam',metrics=['accuracy'])

#fitting the model
model.fit(x_train,y_train,batch_size=16, verbose=1,epochs=10, validation_data=(x_test,y_test),shuffle=False)
#saving model
model.save('BrainTumor10Epochs.h5')




Epoch 1/10


c:\anaconda\lib\site-packages\keras\backend.py:5676: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


150/150 [==============================] - 10s 60ms/step - loss: 0.5258 - accuracy: 0.7504 - val_loss: 0.4780 - val_accuracy: 0.7900
Epoch 2/10
150/150 [==============================] - 8s 50ms/step - loss: 0.3797 - accuracy: 0.8371 - val_loss: 0.3531 - val_accuracy: 0.8333
Epoch 3/10
150/150 [==============================] - 8s 52ms/step - loss: 0.3050 - accuracy: 0.8783 - val_loss: 0.2876 - val_accuracy: 0.8683
Epoch 4/10
150/150 [==============================] - 11s 70ms/step - loss: 0.2409 - accuracy: 0.9129 - val_loss: 0.2285 - val_accuracy: 0.9133
Epoch 5/10
150/150 [==============================] - 9s 62ms/step - loss: 0.1738 - accuracy: 0.9375 - val_loss: 0.1675 - val_accuracy: 0.9467
Epoch 6/10
150/150 [==============================] - 10s 64ms/step - loss: 0.1131 - accuracy: 0.9617 - val_loss: 0.1310 - val_accuracy: 0.9550
Epoch 7/10
150/150 [==============================] - 9s 62ms/step - loss: 0.0882 - accuracy: 0.9721 - val_loss: 0.0984 - val_accuracy: 0.9683
Epoch 8

In [13]:
#testing with prediction
from keras.models import load_model

model1=load_model('BrainTumor10Epochs.h5')
image=cv2.imread('C:\\Users\\Ujjwal Shandilya\\ml code\\brain tumor\\prediction\\pred5.jpg')
img=Image.fromarray(image)
img=img.resize((64,64))
img=np.array(img)
img=np.expand_dims(img,axis=0)
result=model.predict(img)
print(result[0][0])


1/1 [==============================] - 0s 87ms/step
1.0
